# Load the necessary modules

In [1]:
import os
import logging
from dotenv import load_dotenv

from agent_framework.azure import AzureOpenAIChatClient
from agent_framework import ChatAgent

from tools import SqlDatabase

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Read the necessary environment variables

In [2]:
logger.info("Hello from agent-framework-minimal-sql!")

load_dotenv()

# necessary to connect to the Azure OpenAI models
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_MODEL")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")


# Necessary to connect to the Azure SQL Database (or the SQL dedicated pool in Azure Synapse)
connection_string = os.environ.get("AZURE_SQL_CONNECTIONSTRING")
if not connection_string or not endpoint or not deployment_name or not api_key:
    raise ValueError("AZURE_SQL_CONNECTIONSTRING environment variable is required")

## Create the necessary classes and objects

In [3]:
load_dotenv()

# necessary to connect to the Azure OpenAI models
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_MODEL")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")


# this will enable the llm connection to Azure OpenAI
llm = AzureOpenAIChatClient(
    endpoint=endpoint or None,
    deployment_name=deployment_name or None,
    api_key=api_key or None,
    api_version=api_version or None,
)

response = await llm.get_response("Hello, world!")  # test the connection
response.text

'Hello! How can I assist you today?'

In [4]:
# import requests
# print(requests.get('https://icanhazip.com').text)


In [5]:
# this class creates the connection and exposes some methods as tools
db_tool = SqlDatabase(connection_string)

# this will enable the llm connection to Azure OpenAI
llm = AzureOpenAIChatClient(
    endpoint=endpoint or None,
    deployment_name=deployment_name or None,
    api_key=api_key or None,
    api_version=api_version or None,
)

# this creates the agent
# An agent can be thought of as a chat client with a set of tools it
# can use to support the user in its response
agent = ChatAgent(
    chat_client=llm,
    name="agent_name",
    instructions="Your instructions here",  # instructions to guide the agent behavior, aka the system prompt
    tools=[
        db_tool.list_tables,
        db_tool.describe_table,
        db_tool.read_query,
    ],  # the tools the agent can use
    temperature=0.2,  # the lower the temperature, the more deterministic the responses
)

# this creates a new chat thread, it is responsible for managing the
# state of the conversation, including the history of messages, and tools used
# If run without a thread, the agent will not be able to remember previous
# messages or rool calls
thread = agent.get_new_thread()

# Run the agent with a query

In [6]:
query = "List the top 5 airlines by total aircraft."
# result = await agent.run(query, thread=thread)
# print(f"Agent response: {result}")

In [7]:
async for msg in agent.run_stream(query, thread=thread):
    print(f"Streamed message: {msg.to_dict()}")

Streamed message: {'type': 'agent_run_response_update', 'contents': [{'type': 'function_call', 'call_id': 'call_Cpa1gTeGr2jZsvDgKHJx8mLG', 'name': 'list_tables', 'arguments': ''}], 'role': {'type': 'role', 'value': 'assistant'}, 'author_name': 'agent_name', 'response_id': 'chatcmpl-CgBlIXhUAWcF4FWTSreJIXXTQCmjE', 'message_id': 'chatcmpl-CgBlIXhUAWcF4FWTSreJIXXTQCmjE', 'created_at': '2025-11-26T09:30:24.000000Z', 'additional_properties': {'system_fingerprint': 'fp_ee1d74bde0', 'logprobs': None}}
Streamed message: {'type': 'agent_run_response_update', 'contents': [{'type': 'function_call', 'call_id': '', 'name': '', 'arguments': '{}'}], 'role': {'type': 'role', 'value': 'assistant'}, 'author_name': 'agent_name', 'response_id': 'chatcmpl-CgBlIXhUAWcF4FWTSreJIXXTQCmjE', 'message_id': 'chatcmpl-CgBlIXhUAWcF4FWTSreJIXXTQCmjE', 'created_at': '2025-11-26T09:30:24.000000Z', 'additional_properties': {'system_fingerprint': 'fp_ee1d74bde0', 'logprobs': None}}
Streamed message: {'type': 'agent_run_